In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [ ]:
class customdataset(Dataset):
    def __init__(self,data) -> None:
        super().__init__()
        self.data = data
    def __len__(self):
        return len(self.data[0])
    def __getitem__(self, index):
        return self.data[index]

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, ff_dim, output_dim, dropout=0.1 ):
        super(TransformerModel,self).__init__()
        self.embedding = nn.Embedding(input_dim, model_dim)
        self.positional_encoding = nn.Parameter(torch.zeros(1, 100, model_dim))
        self.encoder = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead= num_heads,
            dim_feedforward=ff_dim,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(model_dim, output_dim)
        
    def forward(self, x):
        x = self.embedding(x) + self.positional_encoding[:, :x.size(1), :]
        x = self.transformer_encoder(x)
        x = self.fc_out(x[:, -1, :])  # Output from the last token
        return x

In [ ]:
batch_size = 32
epochs = 25
lr = 0.001

In [ ]:
dataset = customdataset(data)
dataloader = DataLoader(dataset, batch_size= batch_size, shuffle= True)

In [ ]:
# Model, Loss, Optimizer
model = StockTransformer(D_MODEL, N_HEADS, N_LAYERS, DROPOUT)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Training Loop
for epoch in range(epochs):
    total_loss = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.unsqueeze(-1), targets.unsqueeze(-1)  # Add feature dim
        tgt_inputs = torch.zeros_like(targets)  # Decoder input (can be improved)
        optimizer.zero_grad()
        outputs = model(inputs, tgt_inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")

print("Training complete.")